In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

#%matplotlib inline

In [2]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop('index',axis=1)

In [3]:
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [4]:
avisos_detalle['nombre_area'].unique().shape

(188,)

In [5]:
avisos_detalle['nombre_area'].unique()

array(['Comercial', 'Salud', 'Transporte', 'Producción', 'Ventas',
       'Atención al Cliente', 'Mantenimiento',
       'Ingeniería  Eléctrica y Electrónica', 'Abastecimiento',
       'Administración', 'Contabilidad', 'Distribución', 'Redes',
       'Soporte Técnico', 'Calidad', 'Oficios y Profesiones',
       'Farmacéutica', 'Liderazgo de Proyecto', 'Telecomunicaciones',
       'Construcción', 'Prácticas Profesionales', 'Call Center',
       'Recursos Humanos', 'Almacén / Depósito / Expedición',
       'Tecnologia / Sistemas', 'Consultorías Comercio Exterior',
       'Selección', 'Comunicacion', 'Administración de Personal',
       'Planeamiento comercial', 'Gastronomia', 'Tesorería', 'Educación',
       'Auditoría', 'Programación', 'Créditos y Cobranzas',
       'Tecnico de Seguros', 'Asistente', 'Facturación', 'Medicina',
       'Telemarketing', 'Ingeniería  Mecánica',
       'Planeamiento económico-financiero', 'Capacitación',
       'Jóvenes Profesionales', 'Recepcionista', 'Secr

In [6]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [7]:
print(avisos_detalle['nombre_area'].value_counts())

Ventas                         9578
Administración                 5819
Tecnologia/Sistemas            5538
Producción                     4387
Logística                      2443
Atención al Cliente            2410
Oficios                        2100
Recursos Humanos               1828
Finanzas                       1717
Ingenieria                     1640
Salud                          1320
Construcción                   1091
Gastronomia                    1090
Marketing                      1065
Otros                          1037
Secretaria                      811
Comercio Exterior               430
Legales                         428
Minería                         223
Gerencia                        191
Seguros                         181
Comunicacion                    156
Educación                       148
Pasantía                        126
Seguridad e Higiene              97
Química                          69
Diseño                           37
Idiomas                     

In [8]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

In [10]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [11]:
avisos_detalle = oneHotEncoding(avisos_detalle,'nombre_area')
avisos_detalle

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,denominacion_empresa,...,Pasantía,Producción,Química,Recursos Humanos,Salud,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,VENTOR,...,0,0,0,0,0,0,0,0,0,1
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Farmacias Central Oeste,...,0,0,0,0,1,0,0,0,0,0
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,FAMITAX SRL,...,0,0,0,0,0,0,0,0,0,0
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Wurth Argentina S.A,...,0,0,0,0,0,0,0,0,0,0
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,ELECTRO OUTLET SRL,...,0,1,0,0,0,0,0,0,0,0
5,1001135716,1,Vendedor Viajante TUCUMAN/SANTIAGO DEL ESTERO,<p>Somos una empresa multinacional que opera e...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Wurth Argentina S.A,...,0,0,0,0,0,0,0,0,0,1
6,1001326344,1,Vendedor Viajante RECONQUISTA/AVELLANEDA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Wurth Argentina S.A,...,0,0,0,0,0,0,0,0,0,1
7,1001455648,1,Vendedor Viajante PILAR / ESCOBAR/TIGRE,<p>Somos una empresa multinacional que opera e...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Wurth Argentina S.A,...,0,0,0,0,0,0,0,0,0,1
8,9240880,1,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,NaN,Full-time,Jefe / Supervisor / Responsable,Agencia Oficial Alejandro Arizaga,...,0,0,0,0,0,0,0,0,0,1
9,1001202662,1,"promotor/a , asesor/a",<p>En Cerantola nos encontramos en la busqueda...,Gran Buenos Aires,NaN,NaN,Full-time,Otro,cerantola,...,0,0,0,0,0,0,0,0,0,0


In [12]:
test_final = pd.read_csv('../data/test_final_100k.csv')
test_final = test_final.drop('id',axis=1)

In [13]:
test_final.head()

,idaviso,idpostulante
0,739260,6M9ZQR
1,739260,6v1xdL
2,739260,ezRKm9
3,758580,1Q35ej
4,758580,EAN4J6


In [14]:
def feature_area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

In [15]:
test_final = feature_area_de_trabajo(test_final)

In [16]:
test_final

,idaviso,idpostulante,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,...,Pasantía,Producción,Química,Recursos Humanos,Salud,Secretaria,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Ventas
0,739260,6M9ZQR,1,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,...,0,0,0,0,0,0,0,0,0,1
1,739260,6v1xdL,1,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,...,0,0,0,0,0,0,0,0,0,1
2,739260,ezRKm9,1,Asistente Comercial,Nuestro cliente es una empresa nac. ubicada en...,Capital Federal,NaN,NaN,Full-time,Jefe / Supervisor / Responsable,...,0,0,0,0,0,0,0,0,0,1
3,758580,1Q35ej,1,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,NaN,Full-time,Otro,...,0,0,0,0,0,0,0,0,1,0
4,758580,EAN4J6,1,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,NaN,Full-time,Otro,...,0,0,0,0,0,0,0,0,1,0
5,758580,8R6pzR,1,Oracle Financials Sr. Application Developer,"\r\nTeleTech, the worlwide leader in outsource...",Capital Federal,NaN,NaN,Full-time,Otro,...,0,0,0,0,0,0,0,0,1,0
6,776420,aZJ2XN,1,Consultores Senior y SemiSr de Procesos,"Somos una Consultora en Management, Transforma...",Capital Federal,NaN,NaN,Full-time,Otro,...,0,0,0,0,0,0,0,0,0,0
7,776420,Nmpo3J,1,Consultores Senior y SemiSr de Procesos,"Somos una Consultora en Management, Transforma...",Capital Federal,NaN,NaN,Full-time,Otro,...,0,0,0,0,0,0,0,0,0,0
8,776420,eVqWar,1,Consultores Senior y SemiSr de Procesos,"Somos una Consultora en Management, Transforma...",Capital Federal,NaN,NaN,Full-time,Otro,...,0,0,0,0,0,0,0,0,0,0
9,820850,6ZBD33,1,Administrativo de RRHH,Importante compañía de packaging interesada en...,Buenos Aires (fuera de GBA),NaN,NaN,Por Contrato,Otro,...,0,0,0,1,0,0,0,0,0,0
